In [53]:
#IMPORTS
import tensorflow as tf
import os
import numpy as np

In [54]:
dataPath = "../trainingData/"

In [55]:
%%time
import json
#imports all the file paths into a 

text = open(dataPath + "ascii/words.txt","r").read().splitlines()
text = text[18:] #first 18 lines is metadata
images = []
for data in text:
    parts = data.split(" ")
    word = parts[-1]
    tmp = parts[0].split("-")
    path = dataPath + "augmentedWords/IAM/" + tmp[0]+"/"+tmp[0]+"-"+tmp[1]+"/"+parts[0]+"/"
    try:
        inDir = os.listdir(path)
    except:
        inDir=[]
    for f in inDir:
        images.append([os.path.join(path+f), word]) #last 2 are width and height ord() to convert to ascii

images = np.array(images)
np.random.shuffle(images)
np.random.shuffle(images)



print (images[0:10])


#around 1 min

[['../trainingData/augmentedWords/IAM/h02/h02-004/h02-004-05-06/image_0_129.jpeg'
  'We']
 ['../trainingData/augmentedWords/IAM/g06/g06-050i/g06-050i-04-01/image_0_4085.jpeg'
  'man']
 ['../trainingData/augmentedWords/IAM/a01/a01-000x/a01-000x-04-06/image_0_9413.jpeg'
  'by']
 ['../trainingData/augmentedWords/IAM/h01/h01-030/h01-030-08-04/image_0_3712.jpeg'
  'reducing']
 ['../trainingData/augmentedWords/IAM/h07/h07-007/h07-007-07-08/image_0_9019.jpeg'
  'these']
 ['../trainingData/augmentedWords/IAM/a05/a05-058/a05-058-01-10/image_0_9940.jpeg'
  'be']
 ['../trainingData/augmentedWords/IAM/e02/e02-025/e02-025-03-01/image_0_9108.jpeg'
  'of']
 ['../trainingData/augmentedWords/IAM/g07/g07-038/g07-038-07-06/image_0_762.jpeg'
  'to']
 ['../trainingData/augmentedWords/IAM/m04/m04-012/m04-012-05-00/image_0_281.jpeg'
  'Dai']
 ['../trainingData/augmentedWords/IAM/g06/g06-037o/g06-037o-02-09/image_0_9003.jpeg'
  'but']]
CPU times: user 1.89 s, sys: 951 ms, total: 2.84 s
Wall time: 2.84 s


In [86]:
from PIL import Image
from tqdm import tqdm
i = 0
pBar = tqdm(len(images))
while i < len(images):
    pBar.update()
    if os.path.exists(images[i][1]): 
        i+=1
    else:
        np.delete(images,i)

0it [00:09, ?it/s]
102it [00:13,  7.70it/s]

KeyboardInterrupt: 

In [56]:
num_classes = 78

characters = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
char_to_num = tf.keras.layers.StringLookup(vocabulary=characters, oov_token="[UNK]", mask_token=None)
num_to_char = tf.keras.layers.StringLookup(vocabulary=characters, invert=True, mask_token=None)
vocab_size = len(char_to_num.get_vocabulary()) + 1  # +1 for CTC blank

In [57]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Reshape, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D


# Load original base model
base_model = tf.keras.models.load_model("vgg-16-1.keras")
base_model.load_weights("savedModels/bestCheckpoint-1.weights.h5")

# Manually reconstruct using the same input
input_tensor = Input(shape=(224, 224, 3), name="input_1")
x = input_tensor

# Copy all layers except last 4
for layer in base_model.layers[:-4]:
    x = layer(x)

# x now has shape (None, 7, 7, 512)
x = Reshape((49, 512))(x)  # (batch, time_steps, features)

# RNN Layers
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)

# Output layer for CTC
output = Dense(vocab_size, activation="softmax")(x)

# Final model
model = Model(inputs=input_tensor, outputs=output)
model.summary()

Model: "functional_82"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)            │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_9 (Reshape)             │ (None, 49, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 49, 256)        │       656,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 49, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 49, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 49, 64)         │        16,448 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,388,544 (58.70 MB)

 Trainable params: 15,388,032 (58.70 MB)

 Non-trainable params: 512 (2.00 KB)

In [58]:
def ctc_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")
    input_len = tf.ones(shape=(batch_len, 1)) * input_length
    label_len = tf.ones(shape=(batch_len, 1)) * label_length
    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_len, label_len)
    return loss

In [59]:
from tensorflow.keras import layers

# Define remaining model inputs
labels = Input(shape=(None,), dtype="int32", name="labels")
input_lengths = Input(shape=(1,), dtype="int32", name="input_lengths")
label_lengths = Input(shape=(1,), dtype="int32", name="label_lengths")

In [ ]:
# --- CTC LOSS LAYER ---
     y_pred, labels, input_len, label_len = args
    return tf.keras.backend.ctc_batch_cost(labels, y_pred, input_len, label_len)

ctc_loss_output = layers.Lambda(ctc_loss_fn, name="ctc_loss")([x, labels, input_lengths, label_lengths])

In [61]:
training_model = Model(
    inputs=[input_tensor, labels, input_lengths, label_lengths],
    outputs=ctc_loss_output,
    name="ctc_model_with_loss"
)

In [66]:
training_model.compile(optimizer=tf.keras.optimizers.Adam(),loss={"ctc_loss": lambda y_true, y_pred: y_pred})



In [73]:
def load_image_and_label(path, label):
    # Read and decode image
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]

    # Convert label to sequence of character indices
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))

    input_len = tf.constant([49], dtype=tf.int32)  # Fixed output time steps after reshape
    label_len = tf.shape(label)[0:1]  # Shape (1,)

    return {
        "input_1": image,
        "labels": label,
        "input_lengths": input_len,
        "label_lengths": label_len
    }, tf.constant(0)

In [ ]:
paths = images[:, 0]
labels = images[:, 1]

dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.padded_batch( #chatgpt because i had errors
    32,
    padded_shapes=(
        {
            "input_1": [224, 224, 3],       # fixed size images
            "labels": [None],               # variable length labels padded to max in batch
            "input_lengths": [1],           # scalar inside a 1-element vector
            "label_lengths": [1],           # scalar inside a 1-element vector
        },
        []  # shape of label=0 constant (dummy)
    )
).prefetch(tf.data.AUTOTUNE)

In [79]:
training_model.fit(dataset, epochs=20)


Epoch 1/20
 7734/10814 ━━━━━━━━━━━━━━━━━━━━ 13:19 260ms/step - loss: nan

2025-06-12 01:42:43.633715: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at ctc_loss_op.cc:216 : INVALID_ARGUMENT: Not enough time for target transition sequence (required: 53, available: 49)14You can turn this error into a warning by using the flag ignore_longer_outputs_than_inputs
2025-06-12 01:42:43.633757: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Not enough time for target transition sequence (required: 53, available: 49)14You can turn this error into a warning by using the flag ignore_longer_outputs_than_inputs
	 [[{{function_node __inference_one_step_on_data_20658}}{{node ctc_model_with_loss_1/ctc_loss_1/CTCLoss}}]]
	 [[StatefulPartitionedCall/ctc_model_with_loss_1/ctc_loss_1/CTCLoss/_78]]
2025-06-12 01:42:43.633768: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1243263350888270507
2025-06-12 01:42:43.633776: I tensorflow/core/framewo

InvalidArgumentError: Graph execution error:

Detected at node ctc_model_with_loss_1/ctc_loss_1/CTCLoss defined at (most recent call last):
  File "/usr/lib/python3.12/runpy.py", line 198, in _run_module_as_main

  File "/usr/lib/python3.12/runpy.py", line 88, in _run_code

  File "/home/b1az3ng/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/b1az3ng/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/b1az3ng/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/b1az3ng/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/b1az3ng/.venv/lib/python3.12/site-packages/ipykernel/k

In [78]:
training_model.save("RCNN.keras")

/home/b1az3ng/.venv/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:390: UserWarning: The object being serialized includes a `lambda`. This is unsafe. In order to reload the object, you will have to pass `safe_mode=False` to the loading function. Please avoid using `lambda` in the future, and use named Python functions instead. This is the `lambda` being serialized: training_model.compile(optimizer=tf.keras.optimizers.Adam(),loss={"ctc_loss": lambda y_true, y_pred: y_pred})

  return {key: serialize_keras_object(value) for key, value in obj.items()}


NameError: name 'model' is not defined

a